In [120]:
import pandas as pd
import glob
import time
import fatcat_openapi_client
from fatcat_openapi_client.rest import ApiException

In [79]:
n_lens_files =  len(glob.glob('/Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-*.jsonl'))
lens_files = ['/Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-{}.jsonl'.format(i) for i in range(n_lens_files)]
print(len(lens_files))

20


In [74]:
lens_files[0][:-6]

'/Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-0'

In [56]:
df = pd.read_json(lens_files[0], lines=True)
df = df.head(100)

In [78]:
# Get the DOIs from Lens data and write to a file
def extract_dois(lens_files):
    for file in lens_files:
        print('Getting DOIs from {}'.format(file))
        with open('{}-DOIs.txt'.format(file[:-6]), 'w') as f:
            df = pd.read_json(file, lines=True)
            for paper in df['external_ids']:
                for ext_id in paper:
                    if ext_id['type'] == 'doi':
                        f.write(ext_id.get('value'))
                        f.write('\n')
                    else:
                        pass

extract_dois(lens_files)

Getting DOIs from /Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-0.jsonl
Getting DOIs from /Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-1.jsonl
Getting DOIs from /Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-2.jsonl
Getting DOIs from /Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-3.jsonl
Getting DOIs from /Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-4.jsonl
Getting DOIs from /Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-5.jsonl
Getting DOIs from /Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-6.jsonl
Getting DOIs from /Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-7.jsonl
Getting DOIs from /Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-8.jsonl
Getting DOIs from /Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-9.jsonl
Getting DOIs from /Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-10.jsonl
Getting DOIs from /U

In [97]:
n_doi_files =  len(glob.glob('/Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-*-DOIs.txt'))
doi_files = ['/Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-{}-DOIs.txt'.format(i) for i in range(n_lens_files)]
print(len(doi_files))
print(doi_files[0])

20
/Users/timholdsworth/code/lens-lab/Data/lens-mit-papers/export-0-DOIs.txt


In [132]:
conf = fatcat_openapi_client.Configuration()
conf.host = "https://api.fatcat.wiki/v0"
api = fatcat_openapi_client.DefaultApi(fatcat_openapi_client.ApiClient(conf))

In [ ]:
# Get data from the fatcat API using the DOIs
def get_fatcat_data(doi_files):
    for file in doi_files:
        with open(file, 'r') as f: 
            with open(file[:-9] + '-fatcat.jsonl', 'w') as fout:
                dois = f.readlines()
                for doi in dois:
                    time.sleep(.1)
                    try:
                        r = api.lookup_release(doi=doi[:-1])#, expand="files", hide="abstracts,refs")
                        fout.write(str(r))
                        fout.write('\n')
                    except ApiException as ae:
                        if ae.status == 404:
                            print("DOI not found!")
                            #return
                        else:
                            raise ae
    return 'Done'

get_fatcat_data(doi_files)

DOI not found!
